In [14]:
import os
import numpy as np
import librosa
from scipy.io.wavfile import write


In [15]:
data2_path_train = r"C:\Users\admin\Desktop\ARTH SHAH\MGDCC\FoR_norm\for-norm\training"
data2_path_valid = r"C:\Users\admin\Desktop\ARTH SHAH\MGDCC\FoR_norm\for-norm\validation"
data2_path_test = r"C:\Users\admin\Desktop\ARTH SHAH\MGDCC\FoR_norm\for-norm\testing"

data_src_list =  [data2_path_train, data2_path_valid, data2_path_test]  # 

In [16]:
data2_save_path_train = r"C:\Users\admin\Desktop\ARTH SHAH\MGDCC\FoR_norm\noise\hf\10dB\train"
data2_save_path_valid = r"C:\Users\admin\Desktop\ARTH SHAH\MGDCC\FoR_norm\noise\hf\10dB\val"
data2_save_path_test =  r"C:\Users\admin\Desktop\ARTH SHAH\MGDCC\FoR_norm\noise\hf\10dB\test"

data_dst_list = [ data2_save_path_train, data2_save_path_valid, data2_save_path_test]  #[data1_save_path_train, data1_save_path_test,

### Give the value of SNR dB here 

In [17]:
SNR = 10 # desired SNR in dB

In [18]:
noise, noise_sample_rate = librosa.load(r"C:\Users\admin\Desktop\ARTH SHAH\MGDCC\FoR_norm\noise\hf.wav", sr=16000)

### Code to save the noisy audio files 

In [19]:
max_length = 30*16000   #max audio length is 30 seconds


for src_path, dst_path in zip(data_src_list, data_dst_list):

    subfolders = [f.path for f in os.scandir(src_path) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder) 
        subfolder_path = os.path.join(src_path, subfolder_name) 

        save_subfolder_path = os.path.join(dst_path, subfolder_name) 
        os.mkdir(save_subfolder_path)

        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)
            file_name = file

            try:
                audio, sample_rate = librosa.load(file_path, sr=16000)
            except:
                continue

            if len(audio) > max_length:
                audio = audio[:max_length]

            signal_power = np.sum(audio ** 2) / len(audio)
            noise_power = np.sum(noise ** 2) / len(noise)
            noise_amplitude = np.sqrt(signal_power / (noise_power * 10 ** (SNR / 10)))

            try:
                start = np.random.randint(0, noise.shape[0] - audio.shape[0])
            except:
                continue

            noise_segment = noise[start:start + audio.shape[0]]

            audio_with_noise = audio + noise_amplitude * noise_segment

            output_path = os.path.join(save_subfolder_path, file_name)
            write(output_path, sample_rate, audio_with_noise)

            